In [1]:
# Imports
import random
import string
import tempfile
import shutil
import subprocess as sp
import os

from PIL import Image
from torch import nn as nn
from torch import Generator
from torch.utils.data import random_split,DataLoader
# TODO: Currently assumes that pytest runs from project root
from util.car_dataset import check_dir, check_file, CarDataset, DatasetError
from networks.common import *
from networks.CNNbase import CNNBasic

In [2]:
# Helper Functions
def rand_string(length=5):
    return ''.join(random.choices(string.ascii_letters +
        string.digits, k=length))

def loaded_dataset(temp_dir):
    '''
    Loads the car_dataset and returns the CarDataset object.
    '''
    image_folder_list = ['0_Frames','1_Frames','2_Frames','3_Frames']
    label_file_list = ['0.txt','1.txt','2.txt','3.txt']
    #image_folder_list = ['0_Frames']
    #label_file_list = ['0.txt']
    car_data = CarDataset(temp_dir,image_folder_list,label_file_list)
    car_data.drop_nan()

    return car_data

def dataset_loader(loaded_dataset):
    length = len(loaded_dataset)
    split_set = random_split(loaded_dataset,[round(0.80*length)
                ,round(0.20*length)],generator=Generator().manual_seed(42))
    train_loader = DataLoader(split_set[0],batch_size=20)
    test_loader = DataLoader(split_set[1],batch_size=1)
    '''
    Lowering values to sane levels to help run tests on github
    runners. Ideally on local GPU (4GB) 80/20 split with batch size of
    30 works well.
    '''
    return (train_loader,test_loader)

def network():
    network = CNNBasic()
    network = network.to(get_device())
    network.double()
    return network

In [3]:
root = os.getcwd()
print(os.getcwd())

C:\Users\aksha\Desktop\dl-based-sensor-calib


In [4]:
# Load Data
data = loaded_dataset(os.path.join(os.getcwd(),"dummy"))

In [5]:
# Split Dataset
datasets = dataset_loader(data)

In [6]:
# Initialize Network
net = network()

In [7]:
# Train
train = datasets[0]
print("Start Training")
run_training(net,train,1,get_device())

0it [00:00, ?it/s]

Start Training


149it [04:00,  1.62s/it]

Train Epoch: 0	Mean Loss: 0.000578


In [8]:
# Test
test = datasets[1]
print("Start Inference")
run_inference(net,test,get_device())

Start Inference
Num = 742
Len = 742
Test Set Average Loss 0.0713
